# User Engagement Analysis Notebook

This notebook provides an overview of a telecom user engagement analysis workflow. It involves data extraction from a PostgreSQL database, clustering, visualization, and insights generation.

---



In [ ]:
import os
from dotenv import load_dotenv
import psycopg2
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from kneed import KneeLocator

In [ ]:
def connect_to_database():
    try:
        conn = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        return conn
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None


In [ ]:
def get_user_engagement_metrics():
    conn = connect_to_database()
    if conn is None:
        return None
    
    query = """
    SELECT 
        "MSISDN/Number" as user_id,
        COUNT(*) as session_frequency,
        SUM("Dur. (ms)") as total_duration,
        SUM("Total DL (Bytes)" + "Total UL (Bytes)") as total_traffic,
        SUM("Social Media DL (Bytes)" + "Social Media UL (Bytes)") as social_media_traffic,
        SUM("Google DL (Bytes)" + "Google UL (Bytes)") as google_traffic,
        SUM("Email DL (Bytes)" + "Email UL (Bytes)") as email_traffic,
        SUM("Youtube DL (Bytes)" + "Youtube UL (Bytes)") as youtube_traffic,
        SUM("Netflix DL (Bytes)" + "Netflix UL (Bytes)") as netflix_traffic,
        SUM("Gaming DL (Bytes)" + "Gaming UL (Bytes)") as gaming_traffic,
        SUM("Other DL (Bytes)" + "Other UL (Bytes)") as other_traffic
    FROM xdr_data
    GROUP BY "MSISDN/Number"
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df
